<a href="https://colab.research.google.com/github/Erech2020/DeepLearning/blob/master/CargarModelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Cargar el modelo y predecir
</h1>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%tensorflow_version 1.x
%cd /content/drive/My Drive/ColabNotebooks
#%ls
%pwd

In [0]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import tensorflow.keras
import csv
import pickle
"""import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model
import matplotlib.pyplot as plt
import keras
import csv
import pickle"""

In [0]:
modelo_recup = load_model('./VGG16/modelo.h5')
modelo_recup.load_weights('./VGG16/pesos.h5')

labels_recup=list()
archivo = open("./VGG16/labels.txt", "r") 
for linea in archivo.readlines():
  labels_recup.append(linea[:-1]) 
archivo.close()

with open('./VGG16/mod_compi_history.txt', 'rb') as f:  
  modeloEntrenado_history = pickle.load(f)

In [0]:
modelo_recup.summary()

In [0]:
from tensorflow.keras import backend as k
np.random.seed(2)
ruta_data_validacion = './data/validacion'
longitud, altura = 224,224  
nclases = 242

In [0]:
def nombre_corto (nombres):
    nombres_cortos=list()
    for nombre in nombres:
        cad=""
        vacio=0
        for i in nombre[4:]:
            if(i!="_" or i!=" " and vacio<=2):
                
                cad=cad+i
                if(vacio==2):
                    cad=cad+"."
                    break
            if(i=="_" or i==" "):
                vacio=vacio+1 
        nombres_cortos.append(cad)
    return nombres_cortos

In [0]:
def predicciones_nombres(lista_predicciones,lista_etiquetas,valor_minimo=0.3):
    nombre_01="No identificado"
    nombre_02="No identificado"
    da01=0.0
    da02=0.0
    for i,data in enumerate(lista_predicciones):
        if(data>=valor_minimo):
            if(data>=da01):#intercambio de variables
                aux01=da01
                da01=data
                aux02=nombre_01
                nombre_01=lista_etiquetas[i] 
                da02=aux01
                nombre_02=aux02
    return (nombre_01,da01),(nombre_02,da02)

In [0]:
import matplotlib.pyplot as plt

image_path = './VGG16/Negativos/UNAMBA_100687.jpeg'#Ojo con Lena, IMG_20181225_151628.jpg
imagen=load_img(image_path, target_size=(longitud, altura),color_mode="rgb") #"grayscale"   
#x.show() #Imprime el numero en una ventana

plt.imshow(imagen, cmap='gray_r')
plt.axis('off')

imagen=img_to_array(imagen)
imagen=np.expand_dims(imagen,axis=0)
imagen /= 255.

arreglo= modelo_recup.predict(imagen)#PREDICCION
prediccion_array=list(arreglo[0])

print(prediccion_array)
print("Minimo: ",min(prediccion_array))
print("Máximo: ",max(prediccion_array))
n01,n02 = predicciones_nombres(prediccion_array,labels_recup,valor_minimo=0.05)

print('Etiquetas: ',labels_recup)

print("Primera prediccion: ",n01[0], " al ",n01[1]*100,"%")
print("Segunda Prediccion: ",n02[0], " al ",n02[1]*100,"%")

In [0]:
import os
import os.path
direccion='./VGG16/Negativos/'
listaArchivos=os.listdir(direccion)
#print(listaArchivos)
falsos_positivos=0
menor=100
for i, item in enumerate(listaArchivos):
  ruta=direccion+'/'+item
  imagen=load_img(ruta, target_size=(longitud, altura),color_mode="rgb")    
  imagen=img_to_array(imagen)
  imagen=np.expand_dims(imagen,axis=0)
  imagen /= 255.
  arreglo= modelo_recup.predict(imagen)#PREDICCION
  prediccion_array=list(arreglo[0])
  n01,n02 = predicciones_nombres(prediccion_array,labels_recup,valor_minimo=0.05)
  print(item)
  print("Primera prediccion: ",n01[0], " al ",n01[1]*100,"%")
  print("Segunda Prediccion: ",n02[0], " al ",n02[1]*100,"%")
  print(i," ======================")
  if n01[1] < 0.15 :
    falsos_positivos = falsos_positivos +1
  if n01[1] < menor:
    menor=n01[1]
print("Falsos positivos: ",falsos_positivos, "de: ",len(listaArchivos), " Negativos")
print("El menor valor de todos: ",menor*100," %")

<h1>Procesamiento de datos y gráficas </h1>

In [0]:
epocas_ =14  #epocas entrenadas
plt.figure(0)
plt.plot(modeloEntrenado_history['acc'],'r')
plt.plot(modeloEntrenado_history['val_acc'],'g')
plt.xticks(np.arange(0, epocas_, 1.0)) #numero de epocas
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Accuracy")
plt.title("Training Accuracy vs Validation Accuracy")
plt.legend(['train','validation'])

plt.figure(1)
plt.plot(modeloEntrenado_history['loss'],'r')
plt.plot(modeloEntrenado_history['val_loss'],'g')
plt.xticks(np.arange(0, epocas_, 1.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Loss")
plt.title("Training Loss vs Validation Loss")
plt.legend(['train','validation'])

plt.figure(2)
plt.plot(modeloEntrenado_history['mean_squared_error'],'r')
plt.plot(modeloEntrenado_history['val_mean_squared_error'],'g')
plt.xticks(np.arange(0, epocas_, 1.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("MSE")
plt.title("Mean Squared Error vs Validation Mean Squared Error")
plt.legend(['train','validation'])
 
plt.show()

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
validacion_datagen = ImageDataGenerator(rescale = 1. / 255)

validacion_generador = validacion_datagen.flow_from_directory(
    ruta_data_validacion,     #ruta de la carpeta de imagenes validacion
    target_size=(altura, longitud),
    color_mode='rgb',#'grayscale'
    batch_size=1,#batch_size,
    class_mode='categorical',  #None, para devolver solo las imágenes
    shuffle = False,           #se requiere mostrar en orden las imagenes de salida
    seed = 42) 

In [0]:
pred = modelo_recup.predict(validacion_generador, steps=None, max_queue_size=10, workers=200, use_multiprocessing=True, verbose=1)
predicted = np.argmax(pred, axis=1)

In [0]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, recall_score
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

x=validacion_generador.classes                         
cm = confusion_matrix(x, predicted)
accurancy=accuracy_score(x,predicted)
recall=recall_score(x, predicted, average='macro')
print("Accurancy: ",accurancy)
print("Recall: ",recall)

In [0]:
nombre_clases = nombre_corto(labels_recup)
df_cm = pd.DataFrame(cm,nombre_clases, nombre_clases) 
#plt.figure(figsize = (60,60))  #tamaño de la imagen longitud y altura
#sn.set(font_scale=0.8) # tamaño de las clases
#grafica=sn.heatmap(df_cm,cmap='coolwarm',annot=True, annot_kws={"size": 9},cbar=False)
#grafica.set(xlabel='Predicciones',ylabel='Datos Originales')
#plt.show()

In [0]:
#guardar la matriz de confusion
#df_cm.to_csv('./carpeta/matrizConfusion.csv')# guardar la matriz de conf

In [0]:
import pickle
import numpy as np 
import csv
reporte_clasif=classification_report(validacion_generador.classes, predicted, target_names=nombre_clases,output_dict=True)
#reporte = pd.DataFrame(reporte_clasif,range(242),range(1))#,columns=['Names', 'Precision', 'Recall','F1-Score','Suport']) 
#print(reporte_clasif)

In [0]:
import pandas as pd
#dataform=pd.DataFrame.from_dict(data=reporte_clasif).transpose()
#dataform.to_csv('./carpeta/reporteF.csv', header=reporte_clasif.keys()) 

In [0]:
evaluacionDelModelo=modelo_recup.evaluate_generator(validacion_generador, steps=None, max_queue_size=10, workers=100, use_multiprocessing=True)
print(evaluacionDelModelo)

<h1>Comparar todos los entrenamientos</h1> 

In [0]:
import matplotlib.pyplot as plt

import pickle
with open('./VGG16/mod_compi_history.txt', 'rb') as f:  
  vgg16_history = pickle.load(f)

with open('./VGG16UNAMBA/mod_compi_history.txt', 'rb') as f:  
  vgg16_Unamba_history = pickle.load(f)

with open('./denseNet121/mod_compi_history.txt', 'rb') as f:  
  DenseNet121_history = pickle.load(f)

with open('./DenseNet121UNAMBA/mod_compi_history.txt', 'rb') as f:  
  DenseNet121_Unamba_history = pickle.load(f)

In [0]:
import numpy as np 
plt.figure(0)
plt.plot(vgg16_history['val_acc'],'r')
plt.plot(vgg16_Unamba_history['val_acc'],'b') 
plt.plot(DenseNet121_history['val_acc'],'y')   
plt.plot(DenseNet121_Unamba_history['val_acc'],'g')  

plt.xticks(np.arange(0, 16, 2.0))  
plt.rcParams['figure.figsize'] = (8, 6)  
plt.xlabel("Num of Epochs")  
plt.ylabel("Accuracy")  
plt.title("Models and their workouts")  
plt.legend(['VGG16','VGG16UNAMBA','DenseNet121','DenseNet121UNAMBA'])  